In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

## 1. Define Trivial binary function 

$$ g: \mathcal{D} \longrightarrow \mathbb{R} $$
    
where $\mathcal{D}$ is a combinatorial domain (e.g. a set of graphs or molecule structrues). Let $\mathcal{D} = \{1,2,...,d \}$. 

For some $\delta \in \mathcal{D}$, $g(\delta)$ describes the scalar-valued property of graph (e.g. overall utility of a network of bike stations or the solubility of a drug-like molecule).

We model $f$ as affine-linear in presence of each of the $2^{|\mathcal{D}|}$ possible components in $\mathcal{D}$. In particular, let the dummy variable $x_{i}$ describe if a component is present in the combinatorial structure (e.g. a node in a graph or an element in a molecule). In particular, $x_{i} = \mathbb{1}\{ \delta_{i} \in \delta \}$. Restrict to interactions up to order $2$.

$$ f_{\boldsymbol{\alpha}} = \alpha_{0} + \sum_{j=1}^{d} \alpha_{j} x_{j} + \sum_{i<j}^{d} \alpha_{ij}x_{i}x_{j} $$

with $\boldsymbol{\alpha} = (\alpha_{i}, \alpha_{ij}) \in \mathbb{R}^{p}$, $\alpha_{i} \in \mathbb{R}$ and $p=1+d+\binom{d}{k}$.

Let a sample function $f_{1} = \sum \limits_{k \in 2^{|\mathcal{D}|}} \prod \limits_{i=1}^{d} \alpha_{k_{i}} x_{i}$ to be continued.

In [18]:
n, d = 100, 5

np.random.seed(3567)

# X, y
X = np.random.binomial(n=1, p=0.2, size=n*(d+1)).reshape((n,d+1))


array([[1, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [1, 1, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [0, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1],
       [0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 1],
       [1, 1, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0],
 